In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

In [2]:
import tensorflow as tf
import numpy as np

In [3]:
(X_data, y_data), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
print (X_data.shape)

# reduce the dimention of X_train, X_test
def reduce_D (data):
    return data.astype (np.float32).reshape (-1, 28 * 28) / 255.0
X_data, X_test = reduce_D  (X_data), reduce_D (X_test)
print (X_data.shape, X_test.shape)

y_data = y_data.astype(np.int32)
y_test = y_test.astype(np.int32)


X_data_trimmed = X_data[y_data < 5]
y_data_trimmed = y_data [y_data < 5]

X_test = X_test [y_test < 5]
y_test = y_test [y_test < 5]

print (X_data_trimmed.shape, X_test.shape, y_data_trimmed.shape, y_test.shape)

# Check Y :
# print (y_data.shape)
# def y_preprocess (data):
#     return data.astype (np.int32).reshape (data.shape[0], 1)
# y_data, y_test = y_preprocess (y_data), y_preprocess (y_test)
print (y_data.shape, y_test.shape)

# split the train and validation data:
X_validation, y_validation = X_data_trimmed[:5000], y_data_trimmed[:5000]
X_train, y_train = X_data_trimmed[5000:], y_data_trimmed[5000:]

print (X_validation.shape, X_train.shape, y_validation.shape, y_train.shape, X_test.shape, y_test.shape)

(60000, 28, 28)
(60000, 784) (10000, 784)
(30596, 784) (5139, 784) (30596,) (5139,)
(60000,) (5139,)
(5000, 784) (25596, 784) (5000,) (25596,) (5139, 784) (5139,)


In [4]:
reset_graph()
N_feature = X_test.shape[1]
n_layers = 5
n_neurons = 100
learning_rate = 0.01
n_output = 5

In [5]:
X = tf.placeholder (tf.float32, shape = (None, N_feature), name = 'X')
y = tf.placeholder (tf.int32, shape = (None), name = 'y')

In [6]:
with tf.name_scope ('DNN_chapter11_86'):
    he_init = tf.contrib.layers.variance_scaling_initializer (seed = 1)
    hidden1 = tf.layers.dense (X, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden1')
    hidden2 = tf.layers.dense (hidden1, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden2')
    hidden3 = tf.layers.dense (hidden2, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden3')
    hidden4 = tf.layers.dense (hidden3, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden4')
    hidden5 = tf.layers.dense (hidden4, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden5')
    y_predit = tf.layers.dense (hidden5, n_output, kernel_initializer = he_init, name = 'Output')
    y_proba = tf.nn.softmax (y_predit, name = "y_predit")

In [7]:
with tf.name_scope ('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=y_predit)
    loss = tf.reduce_mean(xentropy, name="loss")

In [8]:
with tf.name_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss, name = 'training_op')

In [9]:
with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(y_predit, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name ='accuracy')

In [10]:
init = tf.global_variables_initializer()
saver = tf.train.Saver ()
graph_path = r"C:\Users\asiaynrf\Desktop\fun\handson_ML\Graph_saver"
save_path = r"C:\Users\asiaynrf\Desktop\fun\handson_ML\model_saver"
n_steps = 20
batch_size = 32
data_size = X_train.shape[0]
n_batch = int (np.ceil (data_size / batch_size))

In [11]:
import time
with tf.Session() as sess:
    init.run()
    total_start_time = time.time()
    for step in range (n_steps):
        start_time = time.time ()
        for i_batch in range (n_batch):
            start = i_batch * batch_size
            if i_batch != n_batch - 1:
                end = start + batch_size
            else:
                end = data_size
            sess.run (training_op, feed_dict = {X: X_train[start: end], y: y_train[start: end]})
        train_score = accuracy.eval (feed_dict = {X: X_train[start: end], y: y_train[start: end]})
        validation_score = accuracy.eval (feed_dict = {X: X_validation, y:y_validation})
        end_time = time.time ()
        run_time = end_time - start_time
        print ('At step ', step, 'the train_score is ', train_score, 'the validation score is ', validation_score, 'this cost', run_time)
        saver.save (sess, save_path + '/curr_model.ckpt')
    total_end_time = time.time ()
    accuracy_val = accuracy.eval (feed_dict = {X: X_test, y: y_test})
    print ('Run', n_steps,  'steps total process time is',  total_end_time - total_start_time)
    print ('the test set accuracy is', accuracy_val)
    writer = tf.summary.FileWriter(graph_path, sess.graph)

At step  0 the train_score is  1.0 the validation score is  0.9646 this cost 2.3209245204925537
At step  1 the train_score is  1.0 the validation score is  0.9722 this cost 2.136904716491699
At step  2 the train_score is  1.0 the validation score is  0.9758 this cost 2.125455617904663
At step  3 the train_score is  1.0 the validation score is  0.9778 this cost 2.1196000576019287
At step  4 the train_score is  1.0 the validation score is  0.9788 this cost 2.1200146675109863
At step  5 the train_score is  1.0 the validation score is  0.9794 this cost 2.1298975944519043
At step  6 the train_score is  1.0 the validation score is  0.9804 this cost 2.133791446685791
At step  7 the train_score is  1.0 the validation score is  0.9814 this cost 2.2253570556640625
At step  8 the train_score is  1.0 the validation score is  0.9816 this cost 2.1265745162963867
At step  9 the train_score is  1.0 the validation score is  0.9826 this cost 2.1544876098632812
At step  10 the train_score is  1.0 the val

In [12]:
with tf.Session () as sess: 
    saver.restore (sess, save_path + '/curr_model.ckpt')
    accuracy_test = accuracy.eval (feed_dict = {X: X_test, y: y_test})

INFO:tensorflow:Restoring parameters from C:\Users\asiaynrf\Desktop\fun\handson_ML\model_saver/curr_model.ckpt


In [13]:
accuracy_test

0.99046504

In [14]:
from tensorflow_graph_in_jupyter import show_graph
show_graph (tf.get_default_graph())

In [40]:
reset_graph ()
my_graph = tf.train.import_meta_graph (save_path + '/curr_model.ckpt.meta')

In [41]:
X = tf.get_default_graph ().get_tensor_by_name ('X: 0')
y = tf.get_default_graph ().get_tensor_by_name ('y: 0')

In [42]:
loss = tf.get_default_graph().get_tensor_by_name("loss/loss:0")
Y_proba = tf.get_default_graph().get_tensor_by_name("DNN_chapter11_86/y_predit:0")
logits = Y_proba.op.inputs[0]
accuracy = tf.get_default_graph().get_tensor_by_name("eval/accuracy:0")
Hiddent5 = tf.get_default_graph().get_tensor_by_name ('DNN_chapter11_86/Hidden5/Elu:0')

In [44]:
# training_op = tf.get_default_graph ().get_operation_by_name ('train/training_op')

In [45]:
(X_data, y_data), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
print (X_data.shape)

# reduce the dimention of X_train, X_test
def reduce_D (data):
    return data.astype (np.float32).reshape (-1, 28 * 28) / 255.0
X_data, X_test = reduce_D  (X_data), reduce_D (X_test)
print (X_data.shape, X_test.shape)


(60000, 28, 28)
(60000, 784) (10000, 784)


In [46]:
X_data2_full = X_data[y_data >= 5]
y_data2_full = y_data[y_data >= 5] - 5
X_test2 = X_test[y_test >= 5]
y_test2 = y_test[y_test >= 5] - 5

X_train2_full, X_valid2_full = X_data2_full [5000:], X_data2_full [:5000]
y_train2_full, y_valid2_full = y_data2_full [5000:], y_data2_full [:5000]

print (X_train2_full.shape, y_train2_full.shape, y_valid2_full.shape)

def sample_n_instances_per_class(X, y, n=100):
    Xs, ys = [], []
    for label in np.unique(y):
        idx = (y == label)
        Xc = X[idx][:n]
        yc = y[idx][:n]
        Xs.append(Xc)
        ys.append(yc)
    return np.concatenate(Xs), np.concatenate(ys)

X_train2, y_train2 = sample_n_instances_per_class(X_train2_full, y_train2_full, n=100)
X_valid2, y_valid2 = sample_n_instances_per_class(X_valid2_full, y_valid2_full, n=30)

print (X_train2.shape, X_valid2.shape)

(24404, 784) (24404,) (5000,)
(500, 784) (150, 784)


In [50]:
learning_rate = 0.01

output_layer_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="Output")
optimizer = tf.train.AdamOptimizer(learning_rate, name="Adam2")
training_op = optimizer.minimize(loss, var_list=output_layer_vars)
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

init = tf.global_variables_initializer()
five_frozen_saver = tf.train.Saver()

In [54]:
import time

n_epochs = 1000
batch_size = 20

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

with tf.Session() as sess:
    init.run()
    saver.restore(sess, save_path + '\curr_model.ckpt')
    for var in output_layer_vars:
        var.initializer.run()

    t0 = time.time()
        
    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train2))
        for rnd_indices in np.array_split(rnd_idx, len(X_train2) // batch_size):
            X_batch, y_batch = X_train2[rnd_indices], y_train2[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={X: X_valid2, y: y_valid2})
        if loss_val < best_loss:
            save_path = five_frozen_saver.save(sess, "./my_mnist_model_5_to_9_five_frozen")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

    t1 = time.time()
    print("Total training time: {:.1f}s".format(t1 - t0))

with tf.Session() as sess:
    five_frozen_saver.restore(sess, "./my_mnist_model_5_to_9_five_frozen")
    acc_test = accuracy.eval(feed_dict={X: X_test2, y: y_test2})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

INFO:tensorflow:Restoring parameters from C:\Users\asiaynrf\Desktop\fun\handson_ML\model_saver\curr_model.ckpt
0	Validation loss: 0.709083	Best loss: 0.709083	Accuracy: 74.00%
1	Validation loss: 0.590747	Best loss: 0.590747	Accuracy: 80.67%
2	Validation loss: 0.460661	Best loss: 0.460661	Accuracy: 83.33%
3	Validation loss: 0.438465	Best loss: 0.438465	Accuracy: 84.00%
4	Validation loss: 0.433811	Best loss: 0.433811	Accuracy: 84.67%
5	Validation loss: 0.425761	Best loss: 0.425761	Accuracy: 85.33%
6	Validation loss: 0.391092	Best loss: 0.391092	Accuracy: 86.00%
7	Validation loss: 0.382906	Best loss: 0.382906	Accuracy: 88.00%
8	Validation loss: 0.381215	Best loss: 0.381215	Accuracy: 87.33%
9	Validation loss: 0.393904	Best loss: 0.381215	Accuracy: 85.33%
10	Validation loss: 0.381841	Best loss: 0.381215	Accuracy: 88.00%
11	Validation loss: 0.409746	Best loss: 0.381215	Accuracy: 86.67%
12	Validation loss: 0.355749	Best loss: 0.355749	Accuracy: 88.00%
13	Validation loss: 0.356785	Best loss: 0

In [55]:
from tensorflow_graph_in_jupyter import show_graph
show_graph (tf.get_default_graph())

In [58]:
hidden5_out = Hiddent5

In [60]:
import time

n_epochs = 1000
batch_size = 20

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty
save_path = r'C:\Users\asiaynrf\Desktop\fun\handson_ML\model_saver'

with tf.Session() as sess:
    init.run()
    saver.restore(sess, save_path + '\curr_model.ckpt')
    for var in output_layer_vars:
        var.initializer.run()

    t0 = time.time()
    
    hidden5_train = hidden5_out.eval(feed_dict={X: X_train2, y: y_train2})
    hidden5_valid = hidden5_out.eval(feed_dict={X: X_valid2, y: y_valid2})
        
    for epoch in range(n_epochs):
        rnd_idx = np.random.permutation(len(X_train2))
        for rnd_indices in np.array_split(rnd_idx, len(X_train2) // batch_size):
            h5_batch, y_batch = hidden5_train[rnd_indices], y_train2[rnd_indices]
            sess.run(training_op, feed_dict={hidden5_out: h5_batch, y: y_batch})
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={hidden5_out: hidden5_valid, y: y_valid2})
        if loss_val < best_loss:
            save_path = five_frozen_saver.save(sess, "./my_mnist_model_5_to_9_five_frozen")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation loss: {:.6f}\tBest loss: {:.6f}\tAccuracy: {:.2f}%".format(
            epoch, loss_val, best_loss, acc_val * 100))

    t1 = time.time()
    print("Total training time: {:.1f}s".format(t1 - t0))

with tf.Session() as sess:
    five_frozen_saver.restore(sess, "./my_mnist_model_5_to_9_five_frozen")
    acc_test = accuracy.eval(feed_dict={X: X_test2, y: y_test2})
    print("Final test accuracy: {:.2f}%".format(acc_test * 100))

INFO:tensorflow:Restoring parameters from C:\Users\asiaynrf\Desktop\fun\handson_ML\model_saver\curr_model.ckpt
0	Validation loss: 0.675391	Best loss: 0.675391	Accuracy: 74.00%
1	Validation loss: 0.504377	Best loss: 0.504377	Accuracy: 82.00%
2	Validation loss: 0.473565	Best loss: 0.473565	Accuracy: 83.33%
3	Validation loss: 0.422239	Best loss: 0.422239	Accuracy: 87.33%
4	Validation loss: 0.414334	Best loss: 0.414334	Accuracy: 86.67%
5	Validation loss: 0.411778	Best loss: 0.411778	Accuracy: 87.33%
6	Validation loss: 0.475024	Best loss: 0.411778	Accuracy: 84.67%
7	Validation loss: 0.385870	Best loss: 0.385870	Accuracy: 86.67%
8	Validation loss: 0.399623	Best loss: 0.385870	Accuracy: 88.00%
9	Validation loss: 0.393843	Best loss: 0.385870	Accuracy: 86.67%
10	Validation loss: 0.407487	Best loss: 0.385870	Accuracy: 86.67%
11	Validation loss: 0.378270	Best loss: 0.378270	Accuracy: 87.33%
12	Validation loss: 0.348807	Best loss: 0.348807	Accuracy: 86.67%
13	Validation loss: 0.370040	Best loss: 0